In [1]:
import pandas as pd
from sqlalchemy import create_engine 
import pymysql.cursors
import os
import getpass

In [2]:
pw = getpass.getpass()

1. Establish a connection between Python and the Sakila database.

In [3]:
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/sakila'
engine = create_engine(connection_string)

2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	- `month`: an integer representing the month for which rental data is to be retrieved.
	- `year`: an integer representing the year for which rental data is to be retrieved.

	The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [4]:
def rentals_month(engine, month, year):
    sql = f'''
     select *
     from sakila.rental
     where {year} = year(rental_date)
     and {month} = month(rental_date)
 '''
    df = pd.read_sql_query(sql, engine)
    return df

3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

	The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".


	*Hint: Consider making use of pandas [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)*

In [5]:
def rental_count_month(df, month, year):
    # group the rentals by customer_id and count the number of rentals for each customer
    rentals_by_customer = df.loc[(df['rental_date'].dt.month == month) & (df['rental_date'].dt.year == year)]\
                          .groupby('customer_id')['rental_id'].count().reset_index()

    # rename the 'rental_id' column to 'rentals_{month}'
    rentals_by_customer = rentals_by_customer.rename(columns={'rental_id': f'rentals_{month}_{year}'})
    return rentals_by_customer

4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 
The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [6]:
def compare_rentals(df1, df2):
    # merge the two dataframes on 'customer_id'
    merged_df = pd.merge(df1, df2, on='customer_id')

    # calculate the difference between the number of rentals in the two months
    merged_df['difference'] = merged_df[df1.columns[1]] - merged_df[df2.columns[1]]

    # return the merged dataframe with the new 'difference' column
    return merged_df

In [7]:
may = rentals_month(engine,5,2005)
customer_rentals_may = rental_count_month(may, 5, 2005)

june = rentals_month(engine,6,2005)
customer_rentals_june = rental_count_month(june, 6, 2005)


comparative = compare_rentals(customer_rentals_may, customer_rentals_june)
comparative

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2,7,-5
1,2,1,1,0
2,3,2,4,-2
3,5,3,5,-2
4,6,3,4,-1
...,...,...,...,...
507,594,4,6,-2
508,595,1,2,-1
509,596,6,2,4
510,597,2,3,-1
